In [1]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2021-11-12 17:58:22.528416: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-12 17:58:22.528450: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Jupyter notebook to build, train, and deply CNN models for TF Lite

In [2]:
filename = "tiny_vgg_q" # Set the name for the model output

early_stop_training = True # Early stop training

#___________________________
training_batch_size = 20
training_epochs = 100
training_patience = 10

#___________________________
quantize_aware_training = True
quantize_training_epochs = 100
quantize_training_patience = 10

target_exponent = 5 # Target exponent bit size for custom floating-point 
target_mantissa = 2 # Target mantissa bit size for custom floating-point

In [3]:
google_colab = False # Set True when using on google colab
if google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive')

CNN ARCHITECTURE:

In [4]:
# define cnn model
def ConvPool_CNN_C():
    model = Sequential()

    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(32, 32, 3)))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (1, 1), activation='relu'))
    model.add(Conv2D(10, (1, 1)))
    model.add(GlobalAveragePooling2D())
    model.add(Activation(activation='softmax'))

    # compile model
    #opt = SGD(learning_rate=0.001, momentum=0.9)
    opt = "adam"
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# define cnn model
def shallow_CNN():
	model = Sequential()
	
	model.add(Conv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [7]:
# define cnn model
def shallow_SeparableCNN():
	model = Sequential()
	
	model.add(SeparableConv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [8]:
# define cnn model
def define_model_CNN():
	return shallow_SeparableCNN()

Quantized aware training method.

In [9]:
import struct

def bin2float(b):
    ''' Convert binary string to a float.

    Attributes:
        :b: Binary string to transform.
    '''
    h = int(b, 2).to_bytes(8, byteorder="big")
    return struct.unpack('>d', h)[0]


def float2bin(f):
    ''' Convert float to 64-bit binary string.

    Attributes:
        :f: Float number to transform.
    '''
    [d] = struct.unpack(">Q", struct.pack(">d", f))
    return f'{d:064b}'

In [10]:
def quantize_float (float_number, exponent_bits, mantissa_bits):
    exponent_sign = 1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    if mantissa_bits < 0:
      mantissa_bits = 0
    if exponent_bits < 0:
      exponent_bits = 0
      
    coefficient = float2bin(float_number)
    sign = int (coefficient[:1], 2)
    if 0 < mantissa_bits:
        custom_mantissa = int (coefficient[12 : 12 + mantissa_bits], 2)
    else:
        custom_mantissa = 0
    residual_mantissa = int (coefficient[12 + mantissa_bits:], 2)
    exponent = int (coefficient[1:12], 2) - 1023

    exponent_full_range = pow(2, exponent_bits - exponent_sign) - 1
    if exponent < - exponent_full_range:
        quantized_value = 0
    elif exponent > exponent_full_range:
        quantized_value = pow(-1, sign) * (1 + (1 - pow(2, - mantissa_bits))) * pow(2, exponent_full_range)
    else:
        if (pow (2, (52 - (mantissa_bits + 1))) - 1) < residual_mantissa:
            custom_mantissa += 1
            if (pow (2, mantissa_bits) - 1) < custom_mantissa:
                custom_mantissa = 0
                exponent += 1
    
        quantized_value = pow(-1, sign) * (1 + custom_mantissa * pow(2, - mantissa_bits)) * pow(2, exponent)
    return quantized_value

def quantize_model(model, exponent_bits, mantissa_bits):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      for index in range(len(layer_weights)):
        matrix = layer_weights[index]
        for weight_index, weight in np.ndenumerate(matrix):
          matrix[weight_index] = quantize_float(weight, target_exponent, target_mantissa)
        print (matrix)
        layer_weights[index] = matrix
      layer.set_weights(layer_weights)

Early stop callback.

In [11]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto', restore_best_weights=True)

In [12]:
mantissa_bits = target_mantissa
exponent_bits = target_exponent
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_train_end(self, logs={}):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_batch_end(self, epoch, logs=None):
    quantize_model(self.model, exponent_bits, mantissa_bits)


In [13]:
if not os.path.exists(filename):
  os.mkdir(filename)

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	f = pyplot.figure()
	f.set_figwidth(10)
	f.set_figheight(20)
	
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	pyplot.savefig(filename + '/' + filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model_CNN()
	# fit model
	history = model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback()])
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
	# Save the entire model to a HDF5 file.
	model.save(filename + "/" + filename + '.h5')

In [14]:
# entry point, run the test harness
# load dataset
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# define model
model = define_model_CNN()
# fit model

2021-11-12 17:59:30.359699: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-12 17:59:30.392250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 17:59:30.392988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 74.65GiB/s
2021-11-12 17:59:30.394062: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-12 17:59:30.394756: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libc

In [14]:
if not early_stop_training:
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1)
else:
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=training_patience, verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[monitor])

# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('Pre-quantization training > %.3f' % (acc * 100.0))

2021-11-11 23:52:43.526102: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/100
2499/2500 [============================>.] - ETA: 0s - loss: 1.9148 - accuracy: 0.2996

2021-11-11 23:54:16.827275: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


2500/2500 [==============================] - 91s 37ms/step - loss: 1.9146 - accuracy: 0.2997 - val_loss: 1.5472 - val_accuracy: 0.4517
Epoch 2/100
2500/2500 [==============================] - 100s 40ms/step - loss: 1.6204 - accuracy: 0.4108 - val_loss: 1.6164 - val_accuracy: 0.4568
Epoch 3/100
2500/2500 [==============================] - 122s 49ms/step - loss: 1.5067 - accuracy: 0.4536 - val_loss: 1.2715 - val_accuracy: 0.5387
Epoch 4/100
2500/2500 [==============================] - 109s 44ms/step - loss: 1.4299 - accuracy: 0.4893 - val_loss: 1.5074 - val_accuracy: 0.4782
Epoch 5/100
2500/2500 [==============================] - 92s 37ms/step - loss: 1.3600 - accuracy: 0.5189 - val_loss: 1.1278 - val_accuracy: 0.6037
Epoch 6/100
2500/2500 [==============================] - 95s 38ms/step - loss: 1.3154 - accuracy: 0.5365 - val_loss: 1.1010 - val_accuracy: 0.6184
Epoch 7/100
2500/2500 [==============================] - 95s 38ms/step - loss: 1.2581 - accuracy: 0.5577 - val_loss: 1.1912 - v

2021-11-12 01:22:51.662514: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


Pre-quantization training > 74.120


In [15]:
# learning curves
summarize_diagnostics(history)

NameError: name 'history' is not defined

In [100]:
if not quantize_aware_training:
    print ("_______ Post-training quantization _______")
    quantize_model(model, exponent_bits, mantissa_bits)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('Post-training quantization > %.3f' % (acc * 100.0))
else:
    print ("_______ Quantize aware training _______")
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=quantize_training_patience, verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(trainX, trainY, epochs=quantize_training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback(), monitor])
    # learning curves
    summarize_diagnostics(history)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('Quantize aware training > %.3f' % (acc * 100.0))

_______ Quantize aware training _______


In [37]:
# Save the entire model to a HDF5 file.
model.save(filename + "/" + filename + '.h5')

_______ LOAD CNN MODEL FOR EVALUATION AND CONVERSION TO TF LITE FLOATING-POINT AND FIXED-POINT _______

In [31]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

img_path = "dog.jpg"
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')
x = x / 255.0

model = load_model(filename + "/" + filename + '.h5')

output_data = model.predict(x)

print('TensorFlow Predicted:', output_data)

print("Done!")

TensorFlow Predicted: [[0.00974922 0.01915544 0.0849871  0.0703998  0.09942796 0.1459774
  0.5163244  0.02028543 0.0208753  0.01281799]]
Done!


In [98]:
def print_model(model):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      for index in range(len(layer_weights)):
        matrix = layer_weights[index]
        for weight_index, weight in np.ndenumerate(matrix):
          matrix[weight_index] = quantize_float(weight, target_exponent, target_mantissa)
        print (matrix)
        layer_weights[index] = matrix
      layer.set_weights(layer_weights)

In [99]:
print_model (model)
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))

[[[[-1.     ]
   [-0.1875 ]
   [-0.21875]]

  [[ 0.3125 ]
   [-0.1875 ]
   [-0.375  ]]

  [[ 0.     ]
   [ 0.     ]
   [ 0.375  ]]]


 [[[ 0.875  ]
   [-0.375  ]
   [-0.625  ]]

  [[-0.625  ]
   [ 0.875  ]
   [ 0.     ]]

  [[ 0.5    ]
   [ 0.     ]
   [ 0.875  ]]]


 [[[ 0.     ]
   [ 0.     ]
   [ 0.625  ]]

  [[ 0.5    ]
   [ 0.875  ]
   [-0.3125 ]]

  [[-0.3125 ]
   [-0.625  ]
   [-0.1875 ]]]]
[[[[ 0.      -0.875    0.625    0.125    1.25     0.      -0.25
    -0.5     -0.25    -0.21875  0.3125   0.3125   1.75     0.875
    -1.      -0.1875  -0.125    0.      -1.5     -0.15625  1.75
    -1.       0.      -1.25     0.375   -0.375    1.      -0.4375
     0.      -0.125    0.3125  -0.625    0.3125   0.       0.
     1.      -0.25     1.25     0.15625  0.     ]
   [ 0.375    0.1875  -0.625   -0.625   -0.1875   0.       0.4375
     0.4375   0.75     0.      -0.21875 -0.75     0.      -0.25
     0.125    0.75     0.75    -1.       0.3125  -0.1875   0.
     0.15625  0.       0.3125  -0.75

In [21]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

if not os.path.exists(filename):
  print("Folder " + filename + " does not exist.")
  exit()

# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(filename + "/" + filename + '.h5')

model.summary()

tf.keras.utils.plot_model(model, filename + "/" + filename + ".png", show_shapes=True, show_layer_names=True, expand_nested=True)

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255 , test_images / 255

train_images = tf.cast(train_images, tf.float32)
cifar_ds = tf.data.Dataset.from_tensor_slices((train_images)).batch(1)
def representative_dataset():
  for input_value in cifar_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset = representative_dataset
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + "_f32" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 32, 32, 40)        187       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 40)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 40)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 40)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 16, 16, 60)        2820      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 60)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 60)          0

2021-11-12 01:46:36.796048: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp5n017j_l/assets


2021-11-12 01:46:44.706374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 01:46:44.716559: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-12 01:46:44.733558: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-12 01:46:44.947134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 01:46:44.953471: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563d716398c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-12 01:46:44.953511: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GeForce GT

Done!


2021-11-12 01:46:45.461392: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:313] Ignored output_format.
2021-11-12 01:46:45.461449: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored drop_control_dependency.
2021-11-12 01:46:45.556964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 01:46:45.558010: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 74.65GiB/s
2021-11-12 01:46:45.558311: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-12 

In [22]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + "_i8" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

INFO:tensorflow:Assets written to: /tmp/tmpok_ev0an/assets


INFO:tensorflow:Assets written to: /tmp/tmpok_ev0an/assets
2021-11-12 01:46:56.350979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 01:46:56.351668: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-12 01:46:56.351777: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-12 01:46:56.352611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 01:46:56.353273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB devi

Done!


In [23]:

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename + "/" + filename + "_i8" + '.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [24]:

interpreter.set_tensor(input_details[0]['index'], x)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print('TensorFlow Lite Predicted:', output_data)

TensorFlow Lite Predicted: [[0.0078125  0.0078125  0.09375    0.13671875 0.1796875  0.15625
  0.33984375 0.05078125 0.015625   0.0078125 ]]
